In [10]:
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import os

# ตั้งค่า device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# โหลดโมเดล MTCNN และ InceptionResnetV1
mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# โฟลเดอร์ที่เก็บรูปถ่ายของนักเรียน
student_images_folder = r"D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\data\test_images"

# ลูปสร้าง embeddings สำหรับนักเรียนแต่ละคน
for student_name in os.listdir(student_images_folder):
    student_folder = os.path.join(student_images_folder, student_name)
    
    # ตรวจสอบว่ามีโฟลเดอร์รูปภาพของนักเรียน
    if os.path.isdir(student_folder):
        embeddings_list = []

        # ลูปสำหรับแต่ละรูปภาพในโฟลเดอร์
        for image_name in os.listdir(student_folder):
            image_path = os.path.join(student_folder, image_name)
            img = Image.open(image_path)

            # ใช้ MTCNN เพื่อตัดภาพใบหน้า
            face = mtcnn(img)

            if face is not None:
                # สร้าง embeddings สำหรับใบหน้านั้น
                face_embedding = resnet(face.unsqueeze(0).to(device)).detach()

                # เก็บ embeddings ไว้ในรายการ
                embeddings_list.append(face_embedding)

        # หาค่าเฉลี่ยของ embeddings (ในกรณีที่มีหลายรูป)
        if embeddings_list:
            student_embedding = torch.mean(torch.stack(embeddings_list), dim=0)
            
            # บันทึก embeddings ลงไฟล์ .pt
            torch.save(student_embedding, f"{student_name}_embedding.pt")
            print(f"Saved embedding for {student_name}")
        else:
            print(f"No face detected for {student_name}")


Saved embedding for Akada Iamong
No face detected for Akaradej Sukchan
Saved embedding for Apiwat Rattanaphan
Saved embedding for Apiwit Buachan
Saved embedding for Ekapoj Suthiwong
No face detected for Karanyapas Srikham
Saved embedding for Nattawat Thakhamho
No face detected for Noppawit Sabai
Saved embedding for Panupong Sitthiprom
Saved embedding for Peerakarn Phraphinyokul
Saved embedding for Phacharadanai Rossoda
No face detected for Phuri Khamfei
Saved embedding for Phurin Rueannimit
Saved embedding for Sittikorn Thongdeenok
Saved embedding for Thanaphum Hengarun
Saved embedding for Thanapon Phetprapai
No face detected for Thanwa Suktham


In [1]:
import os
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch
from PIL import Image, ImageDraw
import cv2
import numpy as np

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(keep_all=True, device=device)  # Initialize MTCNN for face detection

# Define the directory containing images of known faces
known_faces_dir = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\data\test_images_aligned'  # Change this to your directory
known_face_names = []
known_face_embeddings = []

# Loop through each person in the directory
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    
    if os.path.isdir(person_dir):  # Ensure it's a directory
        known_face_names.append(person_name)  # Add person's name

        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = Image.open(image_path).convert('RGB')  # Load and convert image
            image = image.resize((160, 160))  # Resize to (160, 160) for FaceNet
            image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Compute embedding
            with torch.no_grad():
                embedding = model(image_tensor).cpu()
                known_face_embeddings.append(embedding)

# Convert list of embeddings to a tensor for easier comparison later
known_face_embeddings = torch.stack(known_face_embeddings)

# Function to recognize faces
def recognize_face(embedding, known_face_embeddings, known_face_names, threshold=0.8):
    distances = [torch.norm(embedding - known_face_embedding).item() for known_face_embedding in known_face_embeddings]
    min_distance_index = np.argmin(distances)
    if distances[min_distance_index] < threshold:
        return known_face_names[min_distance_index]
    return "Unknown"

# Define the directory containing images to be processed
image_dir = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\data\test_images_aligned\Panupong Sitthiprom'  # Change this to your directory
output_dir = 'processed_images'
os.makedirs(output_dir, exist_ok=True)

# Process each image in the directory
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)
    image = Image.open(image_path).convert('RGB')
    
    # Detect faces
    boxes, _ = mtcnn.detect(image)
    
    # Draw faces and names
    image_draw = image.copy()
    draw = ImageDraw.Draw(image_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)  # Draw rectangle around face
            # Get the face region for recognition
            face_region = np.array(image)[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            face_region = cv2.resize(face_region, (160, 160))  # Resize for FaceNet
            face_tensor = torch.tensor(face_region).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Get embedding and recognize
            with torch.no_grad():
                embedding = model(face_tensor).cpu()
                identity = recognize_face(embedding, known_face_embeddings, known_face_names)
            

            # Draw name on the frame
            text_position = (int(box[0]), int(box[1]) - 10)  # Position to draw the name above the rectangle
            draw.text(text_position, identity, fill=(255, 0, 0))  # Draw name
            
            # Draw name on the image
            #font = cv2.FONT_HERSHEY_SIMPLEX
            #text_position = (int(box[0]), int(box[1]) - 10)  # Position to draw the name above the rectangle
           # cv2.putText(np.array(image_draw), identity, text_position, font, 1.0, (255, 0, 0), 2, cv2.LINE_AA)
            
    # Save the processed image
    output_path = os.path.join(output_dir, image_name)
    image_draw.save(output_path)
    print(f"Saved processed image: {output_path}")

IndexError: list index out of range